In [2]:
from urllib2 import urlopen
from urllib2 import HTTPError, URLError
import re
import urllib2
from bs4 import BeautifulSoup
from time import sleep
import unicodecsv as csv
import os
import random

In [80]:
import pandas as pd

In [82]:
import datetime

In [3]:
def make_soup(url):
    hdr = {"User-Agent": "Mozilla/5.0"}
    return BeautifulSoup(urllib2.urlopen(urllib2.Request(url,headers=hdr)).read(),'lxml')

In [67]:
def get_review(url,startID):
    soup = make_soup(url)
    divtable = soup.find("div", class_ = "review_table")
    reviewlist = divtable.find_all("div", class_ = "row review_table_row")
    completereviewdict = {}
    for x in reviewlist: 
        completereviewdict[str(startID)] = {}
        leftcolumn = x.find("div", class_ = "col-xs-8")
        username = leftcolumn.get_text().strip()
        userid = leftcolumn.find("a", class_ = "bold unstyled articleLink")["href"].split("/")[3]
        rightcolumn = x.find("div", class_ = "col-xs-16")
        date = rightcolumn.find("span", class_ = "fr small subtle").get_text()
        userreview = rightcolumn.find("div", class_ = "user_review").get_text().strip()
        if len(rightcolumn.find_all("span",class_="fl")) > 0:
            starlist = rightcolumn.find_all("span", class_ = "glyphicon glyphicon-star")
            if len(starlist) > 0:
                countstar = len(starlist)
                if rightcolumn.find_all("span",class_="fl")[0].get_text() != ' ':
                    rating = float(countstar) + 0.5
                else:
                    rating = float(countstar)
            else:
                rating = 0.5
        else:
            rating = "NA"
        reviewdict = {"Username":username, "User ID": userid, "Date":date, "User Review":userreview,"Rating":rating}
        completereviewdict[str(startID)].update(reviewdict)
        startID += 1
    return completereviewdict   

In [74]:
bigdict = {}
for x in range(1,52): # Change ending index (exclusive) accordingly
    url = "https://www.rottentomatoes.com/m/hurt_locker/reviews/?page="+str(x)+"&type=user" # Change movie url accordingly
    pageStartID = (x-1)*20 + 1
    try:
        onePageDict = get_review(url,pageStartID)
        print 'Page ' + str(x) + ' processed.'
    except AttributeError:
        print 'Page ' + str(x) + ' not processed.'
        continue
    bigdict.update(onePageDict)
    rdtime = random.uniform(0.5,2)
    sleep(rdtime)

Page 1 processed.
Page 2 processed.
Page 3 processed.
Page 4 processed.
Page 5 processed.
Page 6 processed.
Page 7 processed.
Page 8 processed.
Page 9 processed.
Page 10 processed.
Page 11 processed.
Page 12 processed.
Page 13 processed.
Page 14 processed.
Page 15 processed.
Page 16 processed.
Page 17 processed.
Page 18 processed.
Page 19 processed.
Page 20 processed.
Page 21 processed.
Page 22 processed.
Page 23 processed.
Page 24 processed.
Page 25 processed.
Page 26 processed.
Page 27 processed.
Page 28 processed.
Page 29 processed.
Page 30 processed.
Page 31 processed.
Page 32 processed.
Page 33 processed.
Page 34 processed.
Page 35 processed.
Page 36 processed.
Page 37 processed.
Page 38 processed.
Page 39 processed.
Page 40 processed.
Page 41 processed.
Page 42 processed.
Page 43 processed.
Page 44 processed.
Page 45 processed.
Page 46 processed.
Page 47 processed.
Page 48 processed.
Page 49 processed.
Page 50 processed.
Page 51 processed.


In [75]:
len(bigdict) # Check how many reviews were scraped.

1012

In [77]:
os.chdir("/Users/lulinghuang/Desktop/2017 DSC/sci-fi/scrape_rottentmt/") # Change directory accordingly

In [79]:
with open("test_HurtLocker_1To51_April26.csv", "w") as toWrite: # Change file name accordingly
    writer = csv.writer(toWrite, delimiter=',')
    writer.writerow(["Review ID", "Date", 
    'User ID', 'User Review', 'Username','Rating'])
    for a in bigdict.keys():
        try:
            writer.writerow([a,
                            bigdict[a]["Date"].encode('utf-8'),
                            bigdict[a]["User ID"].encode('utf-8'),
                            bigdict[a]["User Review"].encode('utf-8'),
                            bigdict[a]["Username"].encode('utf-8'),
                            str(bigdict[a]["Rating"]).encode('utf-8')])
        except UnicodeDecodeError as detail:
            print 'UnicodeDecodeError'
            print a
            print detail
            continue
        except AttributeError as detail:
            print 'AttributeError'
            print a
            print detail
            continue
print 'Done writing file.'

Done writing file.


In [90]:
## Creat a txt for each review

In [87]:
df = pd.read_csv('test_HurtLocker_1To51_April26.csv',encoding='utf-8')

In [89]:
for index,row in df.iterrows():
    date = row['Date']
    d = datetime.datetime.strptime(date, '%B %d, %Y')
    newd = d.strftime('%Y-%m-%d')
    reviewid = row['Review ID']
    moview = 'HL'
    filename = moview + '_' + newd + '_' + str(reviewid) + '.txt'
    outpath = os.path.join("/Users/lulinghuang/Desktop/2017 DSC/sci-fi/scrape_rottentmt/HL_1to51_txt/", filename)
    with open(outpath,'w') as f:
        f.write(row['User Review'].encode('utf-8'))